In [150]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import shape

In [151]:
mun_ser = pd.read_csv(r'C:\Users\User\Documents\GitHub\BCG2023\Dados\Views\municipios_sertao.csv').drop(columns='Unnamed: 0')

In [159]:
solos = pd.read_csv(r'C:\Users\User\Documents\GitHub\BCG2023\Dados\Views\brasil_solos_5m_20201104.csv',sep=',')
solos['geometry'] = solos['the_geom'].apply(wkt.loads)
solos = solos[['LEG_SINOT', 'geometry']]
solos = gpd.GeoDataFrame(solos, geometry='geometry')
solos.crs = 'EPSG:4326'
solos = solos.to_crs(epsg=3857)
solos

In [162]:
lim_muns = pd.read_json(r'C:\Users\User\Documents\GitHub\BCG2023\Dados\Views\municipios-poligonos.json')
lim_muns = lim_muns[lim_muns['municipioCodigo'].isin(mun_ser['IBGE7'].unique())]
lim_muns['poligono'] = [str(polygon) for polygon in lim_muns['poligono']]
lim_muns['geometry'] = lim_muns['poligono'].apply(lambda x: shape(eval(x)))
lim_muns = lim_muns[['municipioCodigo', 'geometry']]
lim_muns.columns = ['IBGE7', 'geometry']
lim_muns = gpd.GeoDataFrame(lim_muns, geometry='geometry')
lim_muns.crs = 'EPSG:4326'
lim_muns = lim_muns.to_crs(epsg=3857)
lim_muns

,IBGE7,geometry
17,2207959,"MULTIPOLYGON (((-4657602.920 -906816.522, -466..."
18,2207934,"MULTIPOLYGON (((-4700514.765 -882899.906, -469..."
20,2209302,"MULTIPOLYGON (((-4883500.816 -1008107.393, -48..."
33,2203354,"MULTIPOLYGON (((-4712063.340 -1026882.967, -47..."
42,2203305,"MULTIPOLYGON (((-4742086.327 -596473.955, -474..."
...,...,...
5411,2933406,"MULTIPOLYGON (((-4562261.192 -1375609.568, -45..."
5419,3157104,"MULTIPOLYGON (((-4454696.533 -1845038.720, -44..."
5420,2919900,"MULTIPOLYGON (((-4324350.305 -1076824.033, -43..."
5422,2929909,"MULTIPOLYGON (((-4654260.748 -1363150.861, -46..."


In [174]:
intersecao_gdf = gpd.overlay(solos, lim_muns, how='intersection')
intersecao_gdf['AreaSoloHectares'] = intersecao_gdf['geometry'].area  # Converter para hectares
merged_data = lim_muns.merge(intersecao_gdf.groupby(['IBGE7', 'LEG_SINOT'])['AreaSoloHectares'].sum().reset_index(), on='IBGE7', how='left')
merged_data.rename(columns={'LEG_SINOT': 'SOLO', 'AreaSoloHectares': 'AREA_TOTAL'}, inplace=True)
tabela_final = merged_data[['IBGE7', 'SOLO', 'AREA_TOTAL']]
tabela_final['AREA_TOTAL'] = tabela_final['AREA_TOTAL']/10000
tabela_final.to_csv(r'C:\Users\User\Documents\GitHub\BCG2023\Dados\Views\solos_municipios.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_9424\445150884.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela_final['AREA_TOTAL'] = tabela_final['AREA_TOTAL']/10000


In [175]:
tabela_final

,IBGE7,SOLO,AREA_TOTAL
0,2207959,LAd - Latossolos Amarelos Distroficos,64739.067171
1,2207959,PVAe - Argissolos Vermelho-Amarelos Eutroficos,4865.805456
2,2207959,RQo - Neossolos Quartzarenicos Orticos,24660.252674
3,2207934,PVAe - Argissolos Vermelho-Amarelos Eutroficos,23145.966454
4,2207934,RQo - Neossolos Quartzarenicos Orticos,65225.223645
...,...,...,...
3396,3161106,CXbd - Cambissolos Haplicos Tb Distroficos,3117.883407
3397,3161106,CXve - Cambissolos Haplicos Ta Eutroficos,67309.755194
3398,3161106,PVe - Argissolos Vermelhos Eutroficos,41230.393102
3399,3161106,RQo - Neossolos Quartzarenicos Orticos,173855.467762
